In [ ]:
import requests
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup
import re
import urllib.request
import pandas as pd
import numpy as np
import time

In [ ]:
def open_url_2_soup(url):
    """
    open webpage and convert to beautiful soup object
    returns: page_soup - soup object
    params: url - url of the page
    
    """
    headers = {'User-Agent': 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'}
    page = requests.get(url,headers = headers,verify = False)
    page_soup = soup(page.content,'lxml')
    temp = page_soup.findAll("div",{"class":"col-sm-9"})
#     print(temp[2].small.text)
    return page_soup

In [1]:
def getUniversityLookup(url):
    """
    get the list of university and it's basic info from ONE page
    returns : univ_df - dataframe containing name,key and url of universities one page at a time
    params: url - url of the page
    
    """
    univ_name = []
    univ_link = []
    univ_soup = open_url_2_soup(url)
    univ_detail_tag = univ_soup.findAll("div",{"class":"col-sm-9 col-xs-12"})
    for tag in univ_detail_tag:
        a = tag.find("a",href=True)
        univ_name.append(a.text)
        univ_link.append("https://yocket.in"+a['href'])
    #data cleaning 
    univ_df = pd.DataFrame({"name":np.asarray(univ_name),"href": np.asarray(univ_link)})
    print(univ_df['name'].str.split('(',n = 1, expand = True))
    univ_df[['name','keys']] = univ_df['name'].str.split('(',n = 1, expand = True)
    univ_df['keys'] = univ_df['keys'].apply(lambda x: x[:-1])
    return univ_df
getUniversityLookup("https://yocket.in/universities?page=5")

NameError: name 'open_url_2_soup' is not defined

In [ ]:
def getUniversityList():
    """
    returns: UniversityList_df - dataframe containg the basic details of top 500 engineering schools
    params: None
    
    """
    UniversityList_df = pd.DataFrame()
    count = 1
    while(count <= 16):
        url = "https://yocket.in/universities?page=" + str(count)
        temp_df = getUniversityLookup(url)
#         print(temp_df)
        UniversityList_df = UniversityList_df.append(temp_df,ignore_index=True)
#         print(UniversityList_df)
        time.sleep(10) 
        count += 1
    return UniversityList_df




# open_url_2_soup("https://yocket.in/applications-admits-rejects/university-of-california-riverside/2")

In [ ]:
def write_2_csv(df,name):
#     write to csv file
    df.to_csv(name + '.csv', encoding='utf-8')

In [ ]:
# generate csv file
write_2_csv(getUniversityList(),'universityList')